kamarei2@illinois.edu

## Adding the Dolfinx library to the colab environment



In [ ]:
!wget "https://fem-on-colab.github.io/releases/fenicsx-install-development-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"

## Importing the required libraries

In [ ]:
from mpi4py import MPI
from dolfinx import mesh, fem, io, plot, nls, log, geometry, la
from dolfinx import cpp as _cpp
from dolfinx import default_real_type
import basix
import dolfinx.fem.petsc
import ufl
import numpy as np
from petsc4py.PETSc import ScalarType
from petsc4py import PETSc
import time
import os

## Setting material properties and geometry dimensions

In [3]:
# Material properties
E, nu = ScalarType(335000), ScalarType(0.25)	                                    #Young's modulus and Poisson's ratio
mu, lmbda, kappa = E/(2*(1 + nu)), E*nu/((1 + nu)*(1 - 2*nu)), E/(3*(1 - 2*nu))
Gc= ScalarType(0.0268)	                                                          #Critical energy release rate



sts, scs= ScalarType(210), ScalarType(2100)	                                      #Tensile strength and compressive strength
shs = (2/3)*sts*scs/(scs-sts)
Wts = sts**2/(2*E)
Whs = shs**2/(2*kappa)


#Irwin characteristic length
lch=3*Gc*E/8/(sts**2)


#The regularization length
eps=0.04

h=eps/5



comm = MPI.COMM_WORLD
comm_rank = MPI.COMM_WORLD.rank
log.set_log_level(log.LogLevel.ERROR)

#Geometry of the single edge notch geometry
ac = 0.1                                                                          #notch length
W, L = 6.0, 20.0                                                                  #making use of symmetry

## Generating the mesh with element size $h=\frac{ϵ}{5}$


In [5]:
domain = mesh.create_rectangle(comm = comm,points=
                             [np.array([0,0]), np.array([W,L])],n=[int(W/(32*h))
                             ,int(L/(32*h))], cell_type=mesh.CellType.triangle
                               , diagonal= mesh.DiagonalType.crossed)


def cell_criterion(x):
    """Given mesh coordinates, return if each point
    satisfies x[1]<lch*4

    :param x: Input coordinates, shape (num_points, 3)
    :returns: Boolean array of shape (num_points, )
    """
    return (x[1]<lch*4)

ir=0
while ir<2:
    domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim)
    cells_local = np.arange(domain.topology.index_map(
        domain.topology.dim).size_local, dtype=np.int32)
    midpoints = dolfinx.mesh.compute_midpoints(
        domain, domain.topology.dim, cells_local).T
    should_refine = np.flatnonzero(cell_criterion(midpoints)).astype(np.int32)
    domain.topology.create_entities(1)
    local_edges = dolfinx.mesh.compute_incident_entities(
        domain.topology, should_refine, domain.topology.dim, 1)
    domain = dolfinx.mesh.refine(domain, local_edges)[0]
    ir+=1


def cell_criterion2(x):
    """Given mesh coordinates, return if each point
    satisfies (x[1]<2.5*eps) & (x[0]<ac+h*8*4) & (x[0]>ac-h*8*4)

    :param x: Input coordinates, shape (num_points, 3)
    :returns: Boolean array of shape (num_points, )
    """
    return (x[1]<2.5*eps) & (x[0]<ac+h*8*4) & (x[0]>ac-h*8*4)

ir=0
while ir<3:
    domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim)
    cells_local = np.arange(domain.topology.index_map(
        domain.topology.dim).size_local, dtype=np.int32)
    midpoints = dolfinx.mesh.compute_midpoints(
        domain, domain.topology.dim, cells_local).T
    should_refine = np.flatnonzero(cell_criterion2(midpoints)).astype(np.int32)
    domain.topology.create_entities(1)
    local_edges = dolfinx.mesh.compute_incident_entities(
        domain.topology, should_refine, domain.topology.dim, 1)
    domain = dolfinx.mesh.refine(domain, local_edges)[0]
    ir+=1


with dolfinx.io.XDMFFile(domain.comm, "refined_mesh.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
with io.XDMFFile(domain.comm, "paraview/2DSENT.xdmf", "w") as file_results:
    file_results.write_mesh(domain)

## Defining the function spaces

In [6]:
# Defining the function spaces
V = fem.functionspace(domain, ("CG", 1, (domain.geometry.dim,)))                  #Function space for u
Y = fem.functionspace(domain, ("CG", 1))                                          #Function space for z

## Setting the Dirichlet part of the boundary conditions

In [7]:
def left(x):
    return np.isclose(x[0], 0)

def front(x):
    return np.isclose(x[0], W)

def top(x):
    return np.isclose(x[1], L)

def bottom(x):
    return (x[1]<1e-4) & (x[0]>ac - 1e-4)

def cracktip(x):
    return (x[1] < 1e-4) & (x[0] > ac - h*8*2) & (x[0] < ac+1e-4)

def righttop(x):
    return (np.abs(x[1]-0) < 1e-4) & (np.abs(x[0]-W) < 1e-4)

def outer(x):
    return (x[1] > L/10)

fdim = domain.topology.dim -1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
front_facets = mesh.locate_entities_boundary(domain, fdim, front)
top_facets = mesh.locate_entities_boundary(domain, fdim, top)
bottom_facets = mesh.locate_entities(domain, fdim, bottom)
cracktip_facets = mesh.locate_entities(domain, fdim, cracktip)
righttop_facets = mesh.locate_entities(domain, 0, righttop)
outer_facets = mesh.locate_entities(domain, fdim, outer)


dofs_righttop = fem.locate_dofs_topological(V.sub(0), 0, righttop_facets)
dofs_bottom = fem.locate_dofs_topological(V.sub(1), fdim, bottom_facets)

dofs_outer = fem.locate_dofs_topological(Y, fdim, outer_facets)
dofs_cracktip = fem.locate_dofs_topological(Y, fdim, cracktip_facets)

bcl = fem.dirichletbc(ScalarType(0), dofs_righttop, V.sub(0))
bct = fem.dirichletbc(ScalarType(0), dofs_bottom, V.sub(1))
bcs = [bcl, bct]


bct_z2 = fem.dirichletbc(ScalarType(0), dofs_cracktip, Y)
bcs_z = [bct_z2]


## Setting the Neumann part of the boundary condition


In [8]:
sigma_analytical = np.sqrt(E*Gc/np.pi/ac)/((0.752+2.02*(ac/W)+0.37*(1-np.sin(np.pi*ac/2/W))**3)*(np.sqrt(2*W/np.pi/ac*np.tan(np.pi*ac/2/W)))/(np.cos(np.pi*ac/2/W)))
sigma_ext = 2*sigma_analytical
class MyExpression:
    def __init__(self):
        self.t = 0.0
        self.sigma = sigma_ext

    def eval(self, x):
        values = np.zeros((2, x.shape[1]))
        values[1,:] = self.sigma*self.t

        return values

se = MyExpression()
se.t = 0
Tf = fem.Function(V)
Tf.interpolate(se.eval)

## Marking the facets and defining trial and test functions

In [9]:
marked_facets = np.hstack([top_facets, bottom_facets, left_facets])
marked_values = np.hstack([np.full_like(top_facets, 1),
                           np.full_like(bottom_facets, 2),
                           np.full_like(left_facets, 3)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, domain.topology.dim -1,
                          marked_facets[sorted_facets],
                          marked_values[sorted_facets])

metadata = {"quadrature_degree": 2}
ds = ufl.Measure('ds', domain=domain,
                 subdomain_data=facet_tag, metadata=metadata)
dS = ufl.Measure("dS", domain=domain, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

# Define functions
du = ufl.TrialFunction(V)                                                         # Incremental displacement
v  = ufl.TestFunction(V)                                                          # Test function for u
u  = fem.Function(V, name="displacement")                                         # Displacement from previous iteration
u_inc = fem.Function(V)
dz = ufl.TrialFunction(Y)                                                         # Incremental phase field
y  = ufl.TestFunction(Y)                                                          # Test function for z
z  = fem.Function(Y, name="phasefield")                                           # Phase field from previous iteration
z_inc = fem.Function(Y)
d = len(u)

## Setting the initial conditions

In [10]:
u.x.array[:] = 0.


z.x.array[:] = 1.


u_prev = fem.Function(V)
u_prev.x.array[:] = u.x.array
z_prev = fem.Function(Y)
z_prev.x.array[:] = z.x.array


y_dofs_top = fem.locate_dofs_topological(V.sub(1), fdim, top_facets)

## Defining a function to evaluate any field in parallel for visualization purposes

In [11]:
def adjust_array_shape(input_array):
    if input_array.shape == (2,):                                                 # Check if the shape is (2,)
        adjusted_array = np.append(input_array, 0.0)                              # Append 0.0 to the array
        return adjusted_array
    else:
        return input_array
bb_tree = geometry.bb_tree(domain, domain.topology.dim)

def evaluate_function(u, x):
    """[summary]
        Helps evaluated a function at a point `x` in parallel
    Args:
        u ([dolfin.Function]): [function to be evaluated]
        x ([Union(tuple, list, numpy.ndarray)]): [point at which to evaluate function `u`]

    Returns:
        [numpy.ndarray]: [function evaluated at point `x`]
    """


    if isinstance(x, np.ndarray):
        # If x is already a NumPy array
        points0 = x
    elif isinstance(x, (tuple, list)):
        # If x is a tuple or list, convert it to a NumPy array
        points0 = np.array(x)
    else:
        # Handle the case if x is of an unsupported type
        points0 = None

    points = adjust_array_shape(points0)

    u_value = []

    cells = []
    # Find cells whose bounding-box collide with the the points
    cell_candidates = geometry.compute_collisions_points(bb_tree, points)
    # Choose one of the cells that contains the point
    colliding_cells = geometry.compute_colliding_cells(domain, cell_candidates, points)

    if len(colliding_cells.links(0)) > 0:
        u_value = u.eval(points, colliding_cells.links(0)[0])
        u_value = domain.comm.gather(u_value, root=0)
    return u_value[0]

## Defining stored energy, strain and stress functions

In [12]:
# Stored energy, strain and stress functions in linear isotropic elasticity (plane stress)

def energy(v):
	  return mu*(ufl.inner(ufl.sym(ufl.grad(v)),ufl.sym(ufl.grad(v))) + ((nu/(1-nu))**2)*(ufl.tr(ufl.sym(ufl.grad(v))))**2 )+ 0.5*(lmbda)*(ufl.tr(ufl.sym(ufl.grad(v)))*(1-2*nu)/(1-nu))**2

def epsilon(v):
	return ufl.sym(ufl.grad(v))

eta = 0.0
# Stored energy density
psi1 = (z**2+eta)*(energy(u))
psi11 = energy(u)
# Total potential energy
Pi = psi1*dx - ufl.dot(Tf, u)*ds(1)
# Compute first variation of Pi (directional derivative about u in the direction of v)
R = ufl.derivative(Pi, u, v)
# Compute Jacobian of R
Jac = ufl.derivative(R, u, du)

## Weak form of PDE for phase field

In [13]:
#Balance of configurational forces PDE
pen=1000*(3*Gc/8/eps)
Wv=pen/2*((abs(z)-z)**2 + (abs(1-z) - (1-z))**2 )*dx

R_z = y*2*z*(psi11)*dx + 3*Gc/8*(-y/eps + 2*eps*ufl.inner(ufl.grad(z),ufl.grad(y)))*dx + ufl.derivative(Wv,z,y)

# Compute Jacobian of R_z
Jac_z = ufl.derivative(R_z, z, dz)

## Defining a class for solving nonlinear PDEs

In [14]:
class NonlinearPDEProblem:
    """Nonlinear problem class for PDEs."""

    def __init__(self, F, u, bc, J):
        self.L = fem.form(F)
        self.a = fem.form(J)
        self.bc = bc

    def form(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    def F(self, x, b):
        """Assemble residual vector."""
        with b.localForm() as b_local:
            b_local.set(0.0)
        fem.petsc.assemble_vector(b, self.L)
        fem.petsc.apply_lifting(b, [self.a], bcs=[self.bc], x0=[x], alpha=-1.0)
        b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, self.bc, x, -1.0)

    def J(self, x, A):
        """Assemble Jacobian matrix."""
        A.zeroEntries()
        fem.petsc.assemble_matrix(A, self.a, bcs=self.bc)
        A.assemble()

    def matrix(self):
        return fem.petsc.create_matrix(self.a)

    def vector(self):
        return fem.petsc.create_vector(self.L)

## Setting the time step parameters

In [15]:
# time-stepping parameters
T=1
Totalsteps=500
startstepsize=1/Totalsteps
stepsize=startstepsize
t=stepsize
step=1
rnorm_stag0 = 1
rnorm_stag = 1


printsteps = 20

## Creating Newton solvers and their customizations

In [16]:
# Create nonlinear problem
problem_u = NonlinearPDEProblem(R, u, bcs, Jac)

# Create Newton solver and solve

solver = _cpp.nls.petsc.NewtonSolver(MPI.COMM_WORLD)
solver.setF(problem_u.F, problem_u.vector())
solver.setJ(problem_u.J, problem_u.matrix())
solver.set_form(problem_u.form)
solver.max_it = 10
solver.error_on_nonconvergence = False
solver.atol = 1.0e-8
solver.rtol = 1.0e-7

ksp1 = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp1.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"


# Create nonlinear problem
problem_z = NonlinearPDEProblem(R_z, z, bcs_z, Jac_z)

# Create Newton solver and solve

solver_z = _cpp.nls.petsc.NewtonSolver(MPI.COMM_WORLD)
solver_z.setF(problem_z.F, problem_z.vector())
solver_z.setJ(problem_z.J, problem_z.matrix())
solver_z.set_form(problem_z.form)
solver_z.max_it = 10
solver_z.error_on_nonconvergence = False
solver_z.atol = 1.0e-8
solver_z.rtol = 1.0e-7


ksp2 = solver_z.krylov_solver
opts = PETSc.Options()
option_prefix = ksp2.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"

## Solving the problem over several time steps via a staggered scheme

In [ ]:
while t-stepsize < T:

    if comm_rank==0:
        print('Step= %d' %step, 't= %f' %t, 'Stepsize= %e' %stepsize)

    se.t = t
    Tf.interpolate(se.eval)
    stag_iter = 1
    rnorm_stag = 1
    while stag_iter<100 and rnorm_stag/rnorm_stag0 > 1e-7:
        start_time=time.time()
        ##############################################################
        # PDE for u
        ##############################################################
        solver.solve(u.x.petsc_vec)
        u.x.scatter_forward()
        ##############################################################
        # PDE for z
        ##############################################################
        solver_z.solve(z.x.petsc_vec)
        z.x.scatter_forward()
        ##############################################################

        zmin = domain.comm.allreduce(np.min(z.x.array), op=MPI.MIN)


        if comm_rank==0:
            print(zmin)

        if comm_rank==0:
            print("--- %s seconds ---" % (time.time() - start_time))

        ###############################################################
        #Residual check for stag loop
        ###############################################################
        b_e = fem.petsc.assemble_vector(fem.form(-R))
        fint=b_e.copy()
        fem.petsc.set_bc(b_e, bcs)

        rnorm_stag=b_e.norm()
        stag_iter+=1


    ########### Post-processing ##############

    u_prev.x.array[:] = u.x.array
    z_prev.x.array[:] = z.x.array

    # Calculate Reaction

    Fx=domain.comm.allreduce(np.sum(fint[y_dofs_top]), op=MPI.SUM)
    z_x = evaluate_function(z, (ac+eps,0.0))



    if comm_rank==0:
        print(Fx)
        print(z_x)
        with open('Alumina_SENT.txt', 'a') as rfile:
            rfile.write("%s %s %s %s\n" % (str(t), str(zmin), str(z_x), str(Fx)))


    if step % printsteps==0:
        file_results.write_function(u, t)
        file_results.write_function(z, t)


    if z_x<0.05 or np.isnan(zmin):
        t1=t
        break

    # time stepping
    step+=1
    t+=stepsize

sigma_critical=t1*sigma_ext
if comm_rank==0:
    with open('Critical_stress.txt', 'a') as rfile:
        rfile.write("Critical stress= %s\n" % (str(sigma_critical)))
    print('Critical stress= %f' %sigma_critical)
